### INSTALLATION

In [1]:
!pip install wandb
!pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.1 MB/s eta 0:00:00


In [2]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove.6B/

--2024-12-14 15:46:26--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-14 15:46:26--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.04MB/s    in 2m 43s  

2024-12-14 15:49:10 (5.04 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B/glove.6B.50d.txt  
  inflating: glove.6B/glove.6B.100d.txt  
  i

### IMPORTING LIBRARIES

In [3]:
import wandb
import nltk
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import reuters
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from nltk import WordNetLemmatizer
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import torch
from torch import optim
from torch import nn

from sklearn.metrics import hamming_loss, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from gensim.models import Word2Vec
import os
from typing import Tuple, List
from functools import partial

from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
from transformers import AdamW

if not sys.warnoptions:
    warnings.simplefilter("ignore")


[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


### EDA

In [4]:
docs = nltk.corpus.reuters.fileids()
labels = [nltk.corpus.reuters.categories(doc) for doc in docs]
texts = [nltk.corpus.reuters.raw(doc) for doc in docs]
mlb = MultiLabelBinarizer()
bin_labels = mlb.fit_transform(labels)
label_names = mlb.classes_

In [5]:
df = pd.DataFrame({'text': texts})
for i, label in enumerate(label_names):
    df[label] = bin_labels[:, i]

print(df.head())
labels = df.columns[1:]

                                                text  acq  alum  barley  bop  \
0  ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...    0     0       0    0   
1  CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...    0     0       0    0   
2  JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...    0     0       0    0   
3  THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n  ...    0     0       0    0   
4  INDONESIA SEES CPO PRICE RISING SHARPLY\n  Ind...    0     0       0    0   

   carcass  castor-oil  cocoa  coconut  coconut-oil  ...  sun-oil  sunseed  \
0        0           0      0        0            0  ...        0        0   
1        0           0      0        0            0  ...        0        0   
2        0           0      0        0            0  ...        0        0   
3        0           0      0        0            0  ...        0        0   
4        0           0      0        0            0  ...        0        0   

   tea  tin  trade  veg-oil  wheat  wpi  yen  zinc

In [6]:
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

stemmer = SnowballStemmer("english")

def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

def lemmatize(sentence):
    lemmatizer = WordNetLemmatizer()
    lemSentence = ""
    for word in sentence.split():
        lem = lemmatizer.lemmatize(word)
        lemSentence += lem
        lemSentence += " "
    lemSentence = lemSentence.strip()
    return lemSentence

def preprocess_text(train_df):
    train_df['text'] = train_df['text'].str.lower()
    train_df['text'] = train_df['text'].apply(cleanHtml)
    train_df['text'] = train_df['text'].apply(cleanPunc)
    train_df['text'] = train_df['text'].apply(keepAlpha)
    train_df['text'] = train_df['text'].apply(removeStopWords)
    train_df['text'] = train_df['text'].apply(lemmatize)
    return train_df

def create_glove_embedding_matrix(tokenizer, glove_file_path):
    glove_embeddings = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            glove_embeddings[word] = vector

    vocab_size = len(tokenizer.index_word) + 1
    #File 300 dim expected
    embedding_dim = 300
    glove_embedding_matrix = torch.zeros(vocab_size, embedding_dim)

    unknown_words = 0
    for i in range(1, vocab_size):
        word = tokenizer.index_word[i]
        if word in glove_embeddings.keys():
            glove_embedding_matrix[i] = torch.from_numpy(glove_embeddings[word]).float()
        else:
            unknown_words += 1

    print("GloVe embedding matrix created!")
    print('Vocabulary size: {}'.format(vocab_size))
    print('Total unknown words: {}'.format(unknown_words))

    return glove_embeddings, glove_embedding_matrix

def create_glove_label_embedding(labels, glove_embeddings):
    glove_label_embedding = torch.zeros(len(labels), 300)

    for index, label in enumerate(labels):
        wrds = label.split('_')
        for l in wrds:
            if l in glove_embeddings.keys():
                glove_label_embedding[index] += torch.from_numpy(glove_embeddings[l])
        glove_label_embedding[index] /= len(wrds)

    return glove_label_embedding


#Build adjacency matrix based on Co-Occurencies label
def create_adjacency_matrix_cooccurance(data_label):
  cooccur_matrix = np.zeros((data_label.shape[1], data_label.shape[1]), dtype=float)
  for y in data_label:
      y = list(y)
      for i in range(len(y)):
          for j in range(len(y)):
            #data_label
              if y[i] == 1 and y[j] == 1:
                  cooccur_matrix[i, j] += 1
  row_sums = data_label.sum(axis=0)

  for i in range(cooccur_matrix.shape[0]):
    for j in range(cooccur_matrix.shape[0]):
      if row_sums[i]!=0:
        cooccur_matrix[i][j]=cooccur_matrix[i,j]/row_sums[i]
      else:
        cooccur_matrix[i][j]=cooccur_matrix[i,j]

  return cooccur_matrix

class dataset(Dataset):
  def __init__(self, x, y):
    self.x  = x
    self.y = y
  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]


In [7]:
df = preprocess_text(df)

In [8]:
X = df['text'].values
y = df.iloc[:,1:].values
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=200)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(10788,) (10788, 90)
(8630,) (2158,) (8630, 90) (2158, 90)


### MODEL

In [9]:
class MAGNET(nn.Module):
  def __init__(self, input_size, hidden_size, adjacency, embeddings, heads=4, slope=0.01, dropout=0.5):
    super(MAGNET, self).__init__()
    self.embedding = nn.Embedding.from_pretrained(embeddings)
    self.biLSTM = nn.LSTM(input_size,hidden_size,batch_first=True,bidirectional=True)
    self.adjacency = nn.Parameter(adjacency)
    self.dropout = nn.Dropout(dropout)
    self.edge_weights = nn.Linear(hidden_size*2*2, 1, bias=False)
    self.activation = nn.LeakyReLU(slope)
    self.softmax = nn.Softmax(dim=1)
    self.tanh = nn.Tanh()
    self.heads = heads
    self.transform_dim1 = nn.Linear(input_size, hidden_size*2, bias=False)
    self.transform_dim2 = nn.Linear(hidden_size*2, hidden_size*2, bias=False)
    self.transform_dimensions = [self.transform_dim1, self.transform_dim2]

  def forward(self, token, label_embedding):
      #BILSTM part
      features = self.embedding(token)
      out, (h, _) = self.biLSTM(features)
      embedding = torch.cat([h[-2, :, :], h[-1, :, :]], dim=1)
      embedding = self.dropout(embedding)

      #GAT PART
      for td in self.transform_dimensions: #Two Multiheaded GAT layers
        outputs = []
        for head in range(self.heads):
          label_embed = td(label_embedding)
          n, embed_size = label_embed.shape

          label_embed_combinations = label_embed.unsqueeze(1).expand(-1, n, -1)
          label_embed_combinations = torch.cat([label_embed_combinations, label_embed.unsqueeze(0).expand(n, -1, -1)], dim=2)
          e = self.activation(self.edge_weights(label_embed_combinations).squeeze(2))

          attention_coefficients = self.tanh(torch.mul(e,self.adjacency))

          new_h = torch.matmul(attention_coefficients.to(label_embed.dtype), label_embed)
          outputs.append(new_h)
        outputs = self.activation(torch.mean(torch.stack(outputs, dim=0),dim=0))

        label_embedding = outputs
      attention_features = self.dropout(label_embedding)
      attention_features = attention_features.transpose(0, 1)
      predicted_labels = torch.matmul(embedding, attention_features)
      return predicted_labels

In [10]:
def save_checkpoint(model, optimizer, epoch, loss, ckpt_path):
    """
    Save the best model checkpoint
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, ckpt_path)
    print(f"Saved best model to {ckpt_path}")

def load_checkpoint(model, ckpt_path):
    """
    Load the best model checkpoint
    """
    if not os.path.exists(ckpt_path):
        print("No checkpoint found")
        return model

    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model from epoch {checkpoint['epoch']} with loss {checkpoint['loss']:.5f}")
    return model

In [11]:
ckpt_path = 'MAGNET_best_model_final.pt'

In [12]:
def train(model, X_train, label_embedding, y_train,
          total_epoch=250, batch_size=250, learning_rate=0.001,
          ckpt_path=ckpt_path):

    wandb.init(
        project="magnet-classification",
        config={
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "epochs": total_epoch,
            "architecture": "MAGNET"
        }
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    label_embedding = label_embedding.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_data = DataLoader(dataset(X_train, y_train), batch_size=batch_size)

    best_loss = float('inf')

    for epoch in range(1, total_epoch + 1):
        running_loss = 0
        y_pred = []
        model.train()

        for index, (X, y) in enumerate(train_data):
            optimizer.zero_grad()
            out = model(X.to(device), label_embedding)
            loss = criterion(out, y.to(device).float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            optimizer.step()
            y_pred.append(torch.sigmoid(out.detach()).round().cpu())
            running_loss += loss.item()

        y_pred = torch.vstack(y_pred)
        f1score = f1_score(y_train, y_pred, average='micro')
        hammingloss = hamming_loss(y_train, y_pred)

        # Save best model
        if running_loss < best_loss:
            best_loss = running_loss
            save_checkpoint(model, optimizer, epoch, running_loss, ckpt_path)

        wandb.log({
            "epoch": epoch,
            "loss": running_loss,
            "hamming_loss": hammingloss,
            "micro_f1_score": f1score
        })

        print(f'epoch:{epoch} loss:{running_loss:.5f} hamming_loss:{hammingloss:.5f} micro_f1score:{f1score:.5f}')

    wandb.finish()

### TRAINING

In [14]:
tokenizer = Tokenizer(num_words=20000,oov_token='<UNK>')
tokenizer.fit_on_texts(X_train)
sequences_text_train = tokenizer.texts_to_sequences(X_train)
sequences_text_test = tokenizer.texts_to_sequences(X_test)

X_train = torch.from_numpy(pad_sequences(sequences_text_train, maxlen=70))
X_test = torch.from_numpy(pad_sequences(sequences_text_test, maxlen=70))

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )
print(len(tokenizer.index_word))

torch.Size([8630, 70]) torch.Size([2158, 70]) torch.Size([8630, 90]) torch.Size([2158, 90])
24629


In [15]:
#Load glove embedding
glove_embeddings = {}
with open('glove.6B/glove.6B.300d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split()
        word = parts[0]
        vector = np.array(parts[1:], dtype=np.float32)
        glove_embeddings[word] = vector

VOCAB_SIZE = len(tokenizer.index_word)+1
glove_embedding_matrix = torch.zeros(VOCAB_SIZE, 300)

unk = 0
for i in range(1, VOCAB_SIZE):
  word = tokenizer.index_word[i]
  if word in glove_embeddings.keys():
    glove_embedding_matrix[i] = torch.from_numpy(glove_embeddings[word]).float()
  else:
    unk +=1
print('VOCAB_SIZE : {}'.format(VOCAB_SIZE))
print('TOTAL OF UNKNOWN WORD : {}'.format(unk))

VOCAB_SIZE : 24630
TOTAL OF UNKNOWN WORD : 4664


In [18]:
adj_matrix = create_adjacency_matrix_cooccurance(y_train.numpy())
adj_matrix = torch.tensor(adj_matrix)
adj_matrix

tensor([[1.0000e+00, 5.2910e-04, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0582e-03],
        [2.2222e-02, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         2.2222e-02],
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.0000e+00,
         0.0000e+00],
        [7.4074e-02, 3.7037e-02, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00]], dtype=torch.float64)

In [19]:
glove_label_embedding = torch.zeros(len(labels),300)

for index, label in enumerate(labels):
  wrds = label.split('-')
  for l in wrds:
    if l in glove_embeddings.keys():
        glove_label_embedding[index] +=  torch.from_numpy(glove_embeddings[l])
  glove_label_embedding[index]=glove_label_embedding[index]/len(wrds)

print(glove_label_embedding)

tensor([[ 0.1796, -0.1051, -0.5564,  ..., -0.0633,  0.3732, -0.2873],
        [ 0.1101,  0.4061,  0.2036,  ..., -0.1957, -0.4627,  0.6931],
        [-0.3568, -0.1348,  0.0790,  ..., -0.0384,  0.2948,  0.1996],
        ...,
        [-0.1446,  0.0594, -0.1450,  ..., -0.0334,  0.1966,  0.4136],
        [-0.5990, -0.3234, -0.2749,  ...,  0.6343,  0.5300,  0.0299],
        [-0.4541, -0.1300, -0.5178,  ..., -1.1637, -0.2056, -0.3177]])


In [23]:
model = MAGNET(300, 250, adj_matrix, glove_embedding_matrix,heads=8)
train(model, X_train, glove_label_embedding,y_train,total_epoch=250, ckpt_path=ckpt_path)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Saved best model to MAGNET_best_model_final.pt
epoch:1 loss:6.39439 hamming_loss:0.03530 micro_f1score:0.01735
Saved best model to MAGNET_best_model_final.pt
epoch:2 loss:1.82392 hamming_loss:0.01399 micro_f1score:0.00220
Saved best model to MAGNET_best_model_final.pt
epoch:3 loss:1.63280 hamming_loss:0.01379 micro_f1score:0.00112
Saved best model to MAGNET_best_model_final.pt
epoch:4 loss:1.52110 hamming_loss:0.01376 micro_f1score:0.00688
Saved best model to MAGNET_best_model_final.pt
epoch:5 loss:1.36316 hamming_loss:0.01184 micro_f1score:0.31944
Saved best model to MAGNET_best_model_final.pt
epoch:6 loss:1.22197 hamming_loss:0.00993 micro_f1score:0.48730
Saved best model to MAGNET_best_model_final.pt
epoch:7 loss:1.10257 hamming_loss:0.00893 micro_f1score:0.56201
Saved best model to MAGNET_best_model_final.pt
epoch:8 loss:0.98730 hamming_loss:0.00810 micro_f1score:0.61267
Saved best model to MAGNET_best_model_final.pt
epoch:9 loss:0.92494 hamming_loss:0.00779 micro_f1score:0.63827
S

epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇███
hamming_loss,█▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▅▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
micro_f1_score,▁▂▃▃▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇████████████████████
epoch,250
hamming_loss,0.00049
loss,0.04834
micro_f1_score,0.98206


### EVALUATING


In [24]:
def evaluate(model, X_test, label_embedding, y_test, batch_size=250):
    model.eval()
    test_data = DataLoader(dataset(X_test, y_test), batch_size=batch_size)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    label_embedding = label_embedding.to(device)
    y_pred = []
    test_loss = 0
    criterion = nn.BCEWithLogitsLoss()

    with torch.no_grad():
        for X, y in test_data:
            X = X.to(device)
            y = y.to(device)
            out = model(X, label_embedding)
            loss = criterion(out, y.float())
            test_loss += loss.item()
            y_pred.append(torch.sigmoid(out).round().cpu())

    y_pred = torch.vstack(y_pred)

    # Calculate metrics
    f1_micro = f1_score(y_test, y_pred, average='micro')
    hammingloss = hamming_loss(y_test, y_pred)

    print(f"Test Results:")
    print(f"Loss: {test_loss:.5f}")
    print(f"Micro F1: {f1_micro:.5f}")
    print(f"Hamming Loss: {hammingloss:.5f}")

    return y_pred, test_loss

In [25]:
# Usage:
# First load your saved model if necessary
# model = MAGNET(300, 250, adj_matrix, glove_embedding_matrix,heads=8)
# modelll = load_checkpoint(model, 'MAGNET_best_model_final.pt')
# predictions, test_loss = evaluate(modelll, X_test, glove_label_embedding, y_test)